In [2]:
import pandas as pd
iris = pd.read_csv("D:/No tocar/Lic. Ciencias de la Computacion/Cuarto Semestre/Practica Laboral IA/CODE/iris.csv")

In [8]:
iris.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [ ]:
%matplotlib inline
from calendar import c
import matplotlib.pyplot as plt
pd.plotting.scatter_matrix(iris[["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]],
                           figsize=(10,10),c=iris['target'],alpha=0.6)

In [130]:
df = iris.drop(["sepal length (cm)", "sepal width (cm)","target_name"],axis=1)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=37)
for train_index, test_index in split.split(df, df["target"]):
 strat_train_set = df.loc[train_index]
 strat_test_set = df.loc[test_index]
train_copy = strat_train_set.copy()
test_set = strat_test_set
X_train = train_copy[["petal length (cm)", "petal width (cm)"]]
Y_train = train_copy['target']

In [144]:

X_train_1 = train_copy.iloc[:,0:4]
X_train_1

,petal length (cm),petal width (cm),target
131,6.4,2.0,2
95,4.2,1.2,1
13,1.1,0.1,0
149,5.1,1.8,2
96,4.2,1.3,1
...,...,...,...
123,4.9,1.8,2
54,4.6,1.5,1
80,3.8,1.1,1
36,1.3,0.2,0


In [131]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf = knn_clf.fit(X_train,Y_train)

In [132]:
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_predict
y_train_knn_pred = cross_val_predict(knn_clf, X_train, Y_train, cv=3)
precision_score = precision_score(Y_train, y_train_knn_pred, average=None)
recall_score = recall_score(Y_train,y_train_knn_pred,average=None)
print(f"{precision_score} \n {recall_score}")


[1.   0.95 0.95] 
 [1.   0.95 0.95]


In [133]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, y_train_knn_pred)

array([[40,  0,  0],
       [ 0, 38,  2],
       [ 0,  2, 38]], dtype=int64)

In [138]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict

tree_clf = DecisionTreeClassifier(max_depth=3)
tree_clf = tree_clf.fit(X_train,Y_train)

In [139]:

from sklearn.metrics import precision_score, recall_score
y_train_tree_pred = cross_val_predict(tree_clf, X_train, Y_train, cv=3)
precision_score = precision_score(Y_train, y_train_tree_pred, average=None)
recall_score = recall_score(Y_train,y_train_tree_pred,average=None)
print(f"{precision_score} \n {recall_score}")
confusion_matrix(Y_train,y_train_tree_pred)

[1.         0.95121951 0.97435897] 
 [1.    0.975 0.95 ]


array([[40,  0,  0],
       [ 0, 39,  1],
       [ 0,  2, 38]], dtype=int64)

In [141]:
from sklearn.model_selection import cross_val_score
knn_score = cross_val_score(knn_clf, X_train,Y_train, cv = 5, scoring = 'accuracy')
knn_score

array([0.95833333, 0.91666667, 1.        , 0.95833333, 1.        ])

In [142]:
tree_score = cross_val_score(tree_clf, X_train,Y_train, cv = 5, scoring = "accuracy")
tree_score

array([0.95833333, 0.91666667, 1.        , 0.95833333, 1.        ])

In [152]:
from sklearn.ensemble import RandomForestClassifier
rdf_clf = RandomForestClassifier(n_estimators= 15,max_depth=6,n_jobs=-1)
rdf_clf= rdf_clf.fit(X_train,Y_train)

In [153]:
from sklearn.metrics import precision_score, recall_score
y_train_rdf_pred = cross_val_predict(rdf_clf, X_train, Y_train, cv=3)
precision_score = precision_score(Y_train, y_train_rdf_pred, average=None)
recall_score = recall_score(Y_train,y_train_rdf_pred,average=None)
print(f"{precision_score} \n {recall_score}")
confusion_matrix(Y_train,y_train_rdf_pred)

[1.         0.95121951 0.97435897] 
 [1.    0.975 0.95 ]


array([[40,  0,  0],
       [ 0, 39,  1],
       [ 0,  2, 38]], dtype=int64)

In [155]:
rdf_score = cross_val_score(tree_clf, X_train,Y_train, cv = 5, scoring = "accuracy")
rdf_score.mean()

0.9666666666666668